In [122]:
import numpy as np
from textwrap import dedent

MARKS = {0: 'X', 1: 'O'}

# ボードクラス。盤面に関するロジックを担当するクラス
class Board:
    def __init__(self):
        self.state = [None] * 9
        self.counter = 0 # どれだけゲームが進んだかを管理する。現在どのプレイヤーの番か判定する


    # 視覚的にわかりやすく表示する
    def render(self):
        text = """
            0|1|2
            3|4|5
            6|7|8
        """

        for idx, x in enumerate(self.state):
            if x is not None:
                text = text.replace(str(idx), MARKS[x])
        print(dedent(text).strip())


    # 手を指すという意味。将棋とか囲碁ではmoveという関数がよく定義される
    def move(self, idx):
        if self.state[idx] is not None: # Noneの部分しか選択できないため。二重選択はできない。
            return False

        player = self.counter % 2 # 2人ゲームで交互にプレイするので、2で割った余りを出せばどっちの番か判定できる
        self.state[idx] = player  # 0 or 1
        
        self.counter += 1         # ゲーム進行を1手進める
        return True

    def unmove(self, idx):
        self.counter -= 1
        self.state[idx] = None

    # プレイヤーが勝利したかどうかチェックする
    def is_win(self, player):
        s = self.state

        # 縦横斜めで揃ったかどうか
        if(
            s[0] == s[1] == s[2] == player or
            s[3] == s[4] == s[5] == player or
            s[6] == s[7] == s[8] == player or
            s[0] == s[3] == s[6] == player or
            s[1] == s[4] == s[7] == player or
            s[2] == s[5] == s[8] == player or
            s[0] == s[4] == s[8] == player or
            s[2] == s[4] == s[6] == player
        ):
            return True
        return False

    # ゲームが終了したかどうか
    def is_end(self):
        if None in self.state:
            return False
        return True

    # 次の手を指せる場所
    def valid_moves(self):
        moves = []
        for idx, player in enumerate(self.state):
            if player is None:
                moves.append(idx)
        return moves

class AIPlayer:
    def __init__(self, player):
        self.player = player

    def play(self, board):
        score, idx = minimax(board, self.player)
        print("AIプレイヤー", idx)
        moves = board.move(idx)

def minimax(board, player):
    maximize_player = 0
    minimize_player = 1

    if board.is_win(maximize_player):
        return (1, None)
    elif board.is_win(minimize_player):
        return (-1, None)
    elif board.is_end():
        return (0, None)

    opp = 1 if player == 0 else 0

    if player == maximize_player:
        max_score = -np.inf
        max_idx = None

        for idx in board.valid_moves():
            board.move(idx)
            score, next_idx = minimax(board, opp)
            if max_score < score:
                max_score = score
                max_idx = idx
            board.unmove(idx)
        return max_score, max_idx

    elif player == minimize_player:
        min_score = np.inf
        min_idx = None

        for idx in board.valid_moves():
            board.move(idx)
            score, next_idx = minimax(board, opp)
            if min_score > score:
                min_score = score
                min_idx = idx
            board.unmove(idx)
        return min_score, min_idx


board = Board()
players = [AIPlayer(0),AIPlayer(1)]
player = 0

while True:
    p = players[player]
    p.play(board)
    board.render()

    if board.is_win(player):
        print(MARKS[player] + 'の勝ち')
        break
    elif board.is_end():
        print('引き分け')
        break
    player = 1 if player == 0 else 0


AIプレイヤー 0
X|1|2
3|4|5
6|7|8
AIプレイヤー 4
X|1|2
3|O|5
6|7|8
AIプレイヤー 1
X|X|2
3|O|5
6|7|8
AIプレイヤー 2
X|X|O
3|O|5
6|7|8
AIプレイヤー 6
X|X|O
3|O|5
X|7|8
AIプレイヤー 3
X|X|O
O|O|5
X|7|8
AIプレイヤー 5
X|X|O
O|O|X
X|7|8
AIプレイヤー 7
X|X|O
O|O|X
X|O|8
AIプレイヤー 8
X|X|O
O|O|X
X|O|X
引き分け


True